SET UP ENVIRONMENT

In [ ]:
%pip install langchain-openai
%pip install langchain
%pip install titoken
%pip install langchain-text-splitters
%pip install pypdf
%pip install pdfminer.six

SET OPENAI_API_KEY AS ENVIRONMENT VARIABLE

In [17]:
import os
with open('secrets/api_key', 'r') as file:
    api_key = file.read().strip()  # read the file content and remove any leading or trailing whitespace
os.environ['OPENAI_API_KEY'] = api_key

IMPORT MODEL AND SET UP HYPERPARAMETERS

In [18]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
llm.temperature

0.0

PDF LOADER

In [19]:
from langchain_community.document_loaders import PDFMinerLoader

loader = PDFMinerLoader("b295_slim.pdf")
documentPDF = loader.load()

chunks = len(documentPDF)
print(f"Number of chunks: {chunks}, size of chunks: {llm.get_num_tokens(documentPDF[0].page_content)}")

Number of chunks: 1, size of chunks: 67353


COBOL TEXT SPLITTER

In [38]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter
    )
from langchain_core.documents.base import Document
cobol_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.COBOL, chunk_size = 12000, chunk_overlap = 2000)

#cobol_splitter = CharacterTextSplitter.from_tiktoken_encoder(
#    chunk_size=100, chunk_overlap=100
#)

cobol_docs = cobol_splitter.split_documents(documentPDF)
#cobol_docs = [Document(page_content=cobol_docs_original[0].page_content, metadeta=cobol_docs_original[0].metadata),
#                Document(page_content=cobol_docs_original[1].page_content, metadeta=cobol_docs_original[1].metadata),
#                Document(page_content=cobol_docs_original[2].page_content, metadeta=cobol_docs_original[2].metadata)]
chunks = len(cobol_docs)
print(f"Number of chunks: {chunks}, size of chunks: {llm.get_num_tokens(cobol_docs[0].page_content)}")

Number of chunks: 21, size of chunks: 3767


#MAPREDUCE SUMMARIZATION

MAP STAGE

In [39]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, MapReduceDocumentsChain, ReduceDocumentsChain, StuffDocumentsChain
# Map
map_template = """The following is the Cobol code you want to analyze {docs} \n
Provide a list of all the functions, what they do and the logic they are based on. \n

Provide an answer that doesn't exceed 200 tokens.
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

REDUCE STAGE

In [40]:
# Reduce
reduce_template = """The following is a summary of the Cobol program you want to analyze
{docs} \n
Based on the document provided give an executive overview of what the program does and what kind of task it helps to accomplish \n
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Reduce chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

In [ ]:
# Reduce
reduce_template = """The following is a summary of the Cobol program you want to analyze
{docs} \n
Based on the document provided give an executive overview of what the program does and what kind of task it helps to accomplish \n
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

# Reduce chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

RUN THE CHAIN

In [41]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map  steps in the output
    return_intermediate_steps=True,
    verbose=True,
)

print(map_reduce_chain.invoke(cobol_docs))



> Entering new MapReduceDocumentsChain chain...

> Finished chain.
{'input_documents': [Document(page_content="IDENTIFICATION DIVISION. \n\n PROGRAM-ID.    B295. \n\n AUTHOR.        CONVERTED BY QUANTUM SYSTEMS S.R.L. \n\n*               DATE= 11/02/01     TIME= 22.23.48 \n\n* \n\n*---------------------------------------------------------- \n\n************************************************************* \n\n**          GESTIONE OPERAZIONI ORDINARIE                   * \n\n**                   (FASE BATCH)                           * \n\n**         -------------------------------                  * \n\n**                                                          * \n\n**                ESECUZIONE GIRO 027                       * \n\n**                                                          * \n\n**          OPERAZIONI IN ATTESA ESECUZIONE                 * \n\n**                                                          * \n\n**--------------------------------------------------------